# Case 2: D-Stability

In [ ]:
from probabilistic_library import ReliabilityProject, DistributionType, CompareType, ReliabilityMethod
from ptk import Model, Template, VariationType, AggregationType # probabilistic toolkit wrapper for model connection
import os

### Model

In [ ]:
model = Model()
model.executable_file = "../models/D-Stability/D-Stability Console.exe"
model.input_file = "../models/Stability.stix"
model.arguments = '{input} "{ScenarioIndex=2}" "{CalculationIndex=1}"'

if os.path.exists(model.executable_file):
    print (model.executable_file + " exists")
else:
    print (model.executable_file + " does not exist!")

### Input files (stix, zip)

Print all files in the stix file with `get_files()`:

In [ ]:
shear_strength_template = Template()
model.input_templates.append(shear_strength_template)

for file in shear_strength_template.get_files():
    print(file)

Print all captions in the file `soils` with `get_captions()`:

In [ ]:
shear_strength_template.file = "soils"

for caption in shear_strength_template.get_captions():
    print (caption)

Print all attributes that belong to the caption `Soils-Soil-SuShearStrengthModel` with `get_attributes()`:

In [ ]:
shear_strength_template.caption = "Soils-Soil-SuShearStrengthModel"

for attribute in shear_strength_template.get_attributes():
    print (attribute)

Print all components that belong to the caption `Soils-Soil-SuShearStrengthModel` with `get_components()`:

In [ ]:
for component in shear_strength_template.get_components():
    print (component)

In [ ]:
# what are the captions in the file "states-states"?
# what are the attributes that belong to the caption "StateLines-StateLine-Values-Value-Above"?
# what are the components that belong to the caption "StateLines-StateLine-Values-Value-Above"?

### Templates

In [ ]:
shear_strength_template.attribute = "ShearStrengthRatio"
shear_strength_template.component = "-Name"
shear_strength_template.variation_type = VariationType.value
        
strength_increase_template = Template()
strength_increase_template.file = "soils"
strength_increase_template.caption = "Soils-Soil-SuShearStrengthModel"
strength_increase_template.attribute = "StrengthIncreaseExponent"
strength_increase_template.component = "-Name"
strength_increase_template.variation_type = VariationType.value
model.input_templates.append(strength_increase_template)
        
sf_template = Template()
sf_template.file = "results-bishop-bishopresult"
sf_template.caption = ""
sf_template.attribute = "FactorOfSafety"
sf_template.component = ""
sf_template.aggregation_type = AggregationType.last
model.output_templates.append(sf_template)

In [ ]:
# define a template for the attribute "Pop" in the caption "StateLines-StateLine-Values-Value-Above" in the file "states-states"?

#pop_above_template = Template()
#pop_above_template.file = ""
#pop_above_template.caption = ""
#pop_above_template.attribute = ""
#pop_above_template.component = "-Label"
#pop_above_template.variation_type = VariationType.value
#model.input_templates.append(pop_above_template)

Check if the model is ok:

In [ ]:
if model.is_valid():
    print ("Model is ok")
else:
    print ("Model is not ok")
    for message in model.validate():
        print (message)

### Reliability analysis

Define the reliability project:

In [ ]:
project = ReliabilityProject()
project.model = model

Print the model input variables and their initial value:

In [ ]:
for variable in project.variables:
    print(variable.name + ' = ' + str(variable.mean))

Define the random variables (based on the input parameters):

In [ ]:
project.variables["ScenarioIndex"].distribution = DistributionType.deterministic
project.variables["ScenarioIndex"].mean = 2

project.variables["CalculationIndex"].distribution = DistributionType.deterministic
project.variables["CalculationIndex"].mean = 1

project.variables["Clay, shallow.ShearStrengthRatio"].distribution = DistributionType.log_normal
project.variables["Clay, shallow.ShearStrengthRatio"].mean = 0.3
project.variables["Clay, shallow.ShearStrengthRatio"].deviation = 0.01

project.variables["Clay, shallow.StrengthIncreaseExponent"].distribution = DistributionType.log_normal
project.variables["Clay, shallow.StrengthIncreaseExponent"].mean = 0.95
project.variables["Clay, shallow.StrengthIncreaseExponent"].deviation = 0.025

project.variables["Organic clay.ShearStrengthRatio"].distribution = DistributionType.log_normal
project.variables["Organic clay.ShearStrengthRatio"].mean = 0.29
project.variables["Organic clay.ShearStrengthRatio"].deviation = 0.02

project.variables["Organic clay.StrengthIncreaseExponent"].distribution = DistributionType.log_normal
project.variables["Organic clay.StrengthIncreaseExponent"].mean = 0.9
project.variables["Organic clay.StrengthIncreaseExponent"].deviation = 0.03

In [ ]:
# define variables "Peat, deep.ShearStrengthRatio" (log_normal, mean=0.36, deviation=0.06) and 
# "Peat, deep.StrengthIncreaseExponent" (deterministic, mean=0.9)

# if you have created a template for "Pop", define also variables "SP 2.Pop" (log_normal, mean = 15, deviation = 3.5) and
# "SP 4.Pop" (deterministic, mean = 20)

Include a partial correlation of 0.5 between "Clay, shallow.ShearStrengthRatio" and "Clay, shallow.StrengthIncreaseExponent":

In [ ]:
#project.correlation_matrix["variable_name_1", "variable_name_2"] = 0.5

Define the limit state function, $Z = FoS - 1$:

In [ ]:
#project.limit_state_function.compare_type = CompareType.greater_than
#project.limit_state_function.parameter = "FactorOfSafety"
#project.limit_state_function.critical_value = 3.0

Define the calculation settings, choose the `form` method:

In [ ]:
project.settings.reliability_method = ReliabilityMethod.form
project.settings.relaxation_factor = 0.75
project.settings.maximum_iterations = 10

project.settings.max_parallel_processes = 4 
project.settings.save_convergence = True
project.settings.save_realizations = True

Run the reliability calculations:

In [ ]:
if project.is_valid():
    print('project is valid')
else:
    print('project is not valid')

project.run()
print('run completed')

Get insight into the results:

In [ ]:
# the results are stored in project.design_point, you can print and plot them

project.design_point.print()
project.design_point.plot_alphas()
project.design_point.plot_convergence()
project.design_point.plot_realizations()

In [ ]:
for real in project.design_point.realizations:
    real.print()